# Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# For data preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# For classification model
# 'lr', 'rf', 'lightgbm', 'gbc', 'xgboost'
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

# For deep learning model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import AUC

# For evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import shap

In [ ]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)
# Ignore warnings
warnings.filterwarnings('ignore')


In [ ]:
# set the working directory
os.chdir('C:\\Users\\h2408\\Downloads\\RA\\1_paper_LASI\\data')

# Data Preparation

In [ ]:
# Load data
data = pd.read_csv("derived_df.csv")

# Drop the target variables of other papers
target_vars = ['bmi_underweight', 'bmi_overweight', 'waist_circumference']
######################################
target_var = 'waist_circumference'
######################################          
data.shape

In [ ]:
category_col = [
    'education',
    'state',
    'region',
    'religion',
    'MPCE',
    'working_status',
    'occupation',
    'caste',
    'water',
    'alcohol',
    'activity1',
    'benefit'
    ]

# Convert Type
for col in data.columns:
  if col in category_col:
    data[col] = data[col].astype('category')
  else:
    data[col] = data[col].astype('float')

In [ ]:
data.info()

In [ ]:
# Descriptive statistics data
descriptive_data = data.copy()
# Drop the missing values
descriptive_data = data.dropna()
descriptive_data.shape

In [ ]:
used_data = data.copy()
# Drop the missing values
used_data = used_data.dropna()
# Define X and y
X = used_data.drop(target_vars, axis=1)
y = used_data[target_var]
X.shape, y.shape

In [ ]:
groups = {
        'Overall': slice(None),
        'Scheduled Caste': X['caste'] == 'Scheduled caste',
        'Scheduled Tribe': X['caste'] == 'Scheduled tribe',
        'General': X['caste'] == 'General',
        'Other Backward Class': X['caste'] == 'Other backward class',
        'MPCE 1': X['MPCE'] == 'Lowest',
        'MPCE 2': X['MPCE'] == 'Lower middle',
        'MPCE 3': X['MPCE'] == 'Middle',
        'MPCE 4': X['MPCE'] == 'Upper middle',
        'MPCE 5': X['MPCE'] == 'Highest',
    }

In [ ]:
# Category encoding
X = pd.get_dummies(X, drop_first=True)
# dummy_col = ['education_No', 'state_Chandigarh', 'region_Central', 'religion_Others', 'MPCE_Middle', 'working_status_Never worked', 'occupation_Currently no work', 'caste_General', 'water_other', 'alcohol_abstainer', 'activity1_moderate', 'benefit_non-applicable']
# dummy_col = ['education_No', 'state_Chandigarh', 'region_Central', 'religion_Others', 'working_status_Never worked', 'occupation_Currently no work', 'water_other', 'alcohol_abstainer', 'activity1_moderate', 'benefit_non-applicable']
# X = X.drop(dummy_col, axis=1)
X.shape

In [ ]:
X = X.astype('float32')
y = y.astype('float32')

# Standardization
scaler = StandardScaler()
X_sd = scaler.fit_transform(X)
X = pd.DataFrame(X_sd, columns=X.columns, index=X.index)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

# Descriptive Summary Table

In [ ]:
# Define categories and their subcategories
categories = {
    'age_group': ['45-55', '55-65', '65-75', '75-85', '85+'],
    'gender': descriptive_data['gender'].unique(),
    'state': descriptive_data['state'].unique(),
    'education': descriptive_data['education'].unique(),
    'caste': descriptive_data['caste'].unique(),
    'region': descriptive_data['region'].unique(),
    'MPCE': descriptive_data['MPCE'].unique(),
}

# Create a new column for age group
bins = [45, 55, 65, 75, 85, 150]
labels = ['45-55', '55-65', '65-75', '75-85', '85+']
descriptive_data['age_group'] = pd.cut(descriptive_data['age'], bins=bins, labels=labels, right=False)

# Generate the descriptive_data structure
table = {
    'Category': [],
    'Subcategory': [],
    'Group Size': [],
    'Group Proportion': [],
    'Overall Prediction': descriptive_data[target_var].sum(),
    'Prediction': [],
    'Prediction Proportion': []
}
for category, subcats in categories.items():
    grouped = descriptive_data.groupby(category)
    for subcat in subcats:
        subdata = grouped.get_group(subcat)
        table['Category'].append(category)
        table['Subcategory'].append(subcat)
        table['Group Size'].append(len(subdata))
        table['Group Proportion'].append(len(subdata) / len(descriptive_data))
        table['Prediction'].append(subdata[target_var].sum())
        table['Prediction Proportion'].append(subdata[target_var].sum() / descriptive_data[target_var].sum())

table = pd.DataFrame(table)
table['Group Proportion'] = table['Group Proportion'].map('{:.2%}'.format)
table['Prediction Proportion'] = table['Prediction Proportion'].map('{:.2%}'.format)
# save the table
table.to_csv(f'standardized\\{target_var}\\descriptive_table.csv', index=False)

# Functions

In [ ]:
# Function: Define the function to create the DNN model
def create_dnn_model(dim):
    model = Sequential()
    model.add(Dense(128, input_dim=dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification

    # Compile the model with AUROC as a metric
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='auroc')])
    return model

In [ ]:
# Function: Define the function to create the Fully Connected Network (FCN) model
def create_fcn_model(dim):
    model = Sequential()
    model.add(Dense(128, input_dim=dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification

    # Compile the model with AUROC as a metric
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='auroc')])
    return model

# Models

In [ ]:
#logistic regression
model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_lr.fit(X_train, y_train)

#random forest
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train, y_train)

#XGBoost
model_xgb = XGBClassifier(eval_metric='logloss', random_state=42)
model_xgb.fit(X_train, y_train)

#gradient boosting
model_gb = GradientBoostingClassifier(random_state=42)
model_gb.fit(X_train, y_train)

#lightGBM
model_lgbm = LGBMClassifier(random_state=42, force_row_wise=True, verbose=-1)
model_lgbm.fit(X_train, y_train)

In [ ]:
#DNN
model_dnn = create_dnn_model(X_train.shape[1])
model_dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='auroc')])
model_dnn.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

In [ ]:
#FCN
model_fcn = create_fcn_model(X_train.shape[1])
model_fcn.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='auroc')])
model_fcn.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

In [ ]:
final_models = {
    'Logistic Regression': model_lr,
    'Random Forest': model_rf,
    'XGBoost': model_xgb,
    'Gradient Boosting': model_gb,
    'LightGBM': model_lgbm,
    'DNN': model_dnn,
    'FCN': model_fcn
}

# ROC Curve

In [ ]:
# Plot ROC Curves
plt.figure(figsize=(10, 10))
    
for model_name, model in final_models.items():
    if model_name in ['DNN', 'FCN']:
        y_proba = model.predict(X_test).ravel()
    else:
        y_proba = model.predict_proba(X_test)[:, 1]
    # calculate roc curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    # plot the roc curve for the model
    plt.plot(fpr, tpr, label=f"{model_name} (AUC = {auc:.2f})", linewidth=3)
    
# plot the roc curve for the random model
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guess')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)
plt.title(f'ROC Curves', fontsize=26)
plt.legend(fontsize=20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)

# Save the plot
plt.savefig(f"standardized\\{target_var}\\fig\\roc_curves.png")
plt.show()    

In [ ]:
# Plot ROC Curves
plt.figure(figsize=(10, 10))
    
for model_name, model in final_models.items():
    if model_name in ['DNN', 'FCN']:
        # y_proba = model.predict(X_test).ravel()
        continue
    else:
        y_proba = model.predict_proba(X_test)[:, 1]
    # calculate roc curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    # plot the roc curve for the model
    plt.plot(fpr, tpr, label=f"{model_name} (AUC = {auc:.2f})", linewidth=3)
    
# plot the roc curve for the random model
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guess')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)
plt.title(f'ROC Curves', fontsize=26)
plt.legend(fontsize=20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)

# Save the plot
plt.savefig(f"standardized\\{target_var}\\fig\\roc_curves_2.png")
plt.show()    

# Density Plot

In [ ]:
# Function: Plot the density prediction plot by group
def density_prediction_plot_by_group(model, model_name, X_test, y_test, target_var, groups):
    # Evaluate and Plot for Each Group
    for group_name, group_indices in groups.items():
        X_group = X_test[group_indices]
        y_group = y_test[group_indices]

        # Predict the probabilities
        if model_name in ['DNN', 'FCN']:
            y_pred_proba_group = model.predict(X_group).ravel()
        else:
            y_pred_proba_group = model.predict_proba(X_group)[:, 1]

        # Get the risk scores
        risk_scores = y_pred_proba_group
        risk_scores_neg = risk_scores[y_group == 0]
        risk_scores_pos = risk_scores[y_group == 1]

        # Create DataFrames for plotting
        df_neg = pd.DataFrame({"Predicted Probability": risk_scores_neg, "Label": f"Missing {target_var}"})
        df_pos = pd.DataFrame({"Predicted Probability": risk_scores_pos, "Label": f"Having {target_var}"})

        # Plot the density plot
        plt.figure(figsize=(10, 8))
        ax = plt.gca()
        ax.spines['top'].set_visible(False)  # Hide the top spine
        ax.spines['right'].set_visible(False)  # Hide the right spine
        sns.kdeplot(data=df_neg, x="Predicted Probability", color='orange', fill=True, alpha=0.5)
        sns.kdeplot(data=df_pos, x="Predicted Probability", color='blue', fill=True, alpha=0.5)
        plt.xlabel("Predicted Probability", fontsize=22)
        plt.ylabel("Density", fontsize=22)
        plt.xticks(fontsize=24)
        plt.yticks(fontsize=24)
        
        plt.title(f"{model_name}: {group_name} Density Plot", fontsize=24)
        plt.tight_layout()
        # Save the plot
        plt.savefig(f'standardized\\{target_var}\\fig\\{model_name}_{group_name}.png', pad_inches=0.1, bbox_inches='tight')
        plt.show()

In [ ]:
# for model_name, model in final_models.items():
#     density_prediction_plot_by_group(model, model_name, X_test, y_test, target_var, groups)
density_prediction_plot_by_group(model = model_lgbm, model_name = 'LightGBM', X_test = X_test, y_test = y_test, target_var = target_var, groups = groups)